### Equilibrium Matrix in Markov Chain using Python in Stock Market

In [5]:
!pip install nsepython

In [7]:
from nsepython import *
import datetime

# Transition Matrix

In [8]:
symbol = "NIFTY 50"
days = 1000
end_date = datetime.datetime.now().strftime("%d-%b-%Y")
end_date = str(end_date)

start_date = (datetime.datetime.now()- datetime.timedelta(days=days)).strftime("%d-%b-%Y")
start_date = str(start_date)

df=index_history("NIFTY 50",start_date,end_date)

df["state"]=df["CLOSE"].astype(float).pct_change()
df['state']=df['state'].apply(lambda x: 'Upside' if (x > 0.001) else ('Downside' if (x<=0.001) else 'Consolidation'))

df['priorstate']=df['state'].shift(1)

states = df [['priorstate','state']].dropna()
states_matrix = states.groupby(['priorstate','state']).size().unstack().fillna(0)

transition_matrix= states_matrix.apply(lambda x: x/float(x.sum()),axis=1)
print(transition_matrix)

state          Downside    Upside
priorstate                       
Consolidation  0.000000  1.000000
Downside       0.650367  0.349633
Upside         0.543396  0.456604


## Forecasting Futures Probabilities of States using Python

Let’s consider time as t. Now, We will see observe other probability matrices.“Transition Matrix” is the probability matrix at t=0. It shows the probability at t=0. However, we will build the Markov Chain by multiplying this transition matrix by itself to obtain the probability matrix in t=1 which would allow us to make one-day forecasts.

In [10]:
symbol = "NIFTY 50"
days = 1000
end_date = datetime.datetime.now().strftime("%d-%b-%Y")
end_date = str(end_date)

start_date = (datetime.datetime.now()- datetime.timedelta(days=days)).strftime("%d-%b-%Y")
start_date = str(start_date)

df=index_history("NIFTY 50",start_date,end_date)

df["state"]=df["CLOSE"].astype(float).pct_change()
df['state']=df['state'].apply(lambda x: 'Upside' if (x > 0) else 'Downside' )

df['priorstate']=df['state'].shift(1)

states = df [['priorstate','state']].dropna()
states_matrix = states.groupby(['priorstate','state']).size().unstack().fillna(0)

transition_matrix= states_matrix.apply(lambda x: x/float(x.sum()),axis=1)
print(transition_matrix)

state       Downside    Upside
priorstate                    
Downside    0.598945  0.401055
Upside      0.513514  0.486486


In [11]:
t_0 = transition_matrix.copy()
t_1 =t_0.dot(t_0)
t_1

state,Downside,Upside
priorstate,,
Downside,0.564682,0.435318
Upside,0.557384,0.442616


Now, We need to extend the Markov Chain similarly.If we continue multiplying the transition matrix that we have obtained in t=1 by the original transition matrix in t0, we obtain the probabilities in time t=2. Let’s find the transition matrix at t=2 and t=3 and so on in a similar manner.

In [12]:
t_2 = t_0.dot(t_1)
t_2

state,Downside,Upside
priorstate,,
Downside,0.561755,0.438245
Upside,0.561131,0.438869


In [13]:
t_3 = t_0.dot(t_2)
t_3

state,Downside,Upside
priorstate,,
Downside,0.561505,0.438495
Upside,0.561452,0.438548


## Equilibrium Matrix using Python

In [14]:
## Equilibrium Matrix using Python

t_0 = transition_matrix.copy()

t_m = t_0.copy()
t_n = t_0.dot(t_0)

i = 1
while(not(t_m.equals(t_n))):
    i += 1
    t_m = t_n.copy()
    t_n = t_n.dot(t_0)

print("Equilibrium Matrix Number: " + str(i))
print(t_n)

Equilibrium Matrix Number: 17
state       Downside    Upside
priorstate                    
Downside    0.561481  0.438519
Upside      0.561481  0.438519


The equilibrium Matrix is a stationary state. So, As per the theory of the Markov Chain, This figure will stay the same for foreseeable data points

In [15]:
##Random Walk

symbol = "NIFTY 50"
days = 9209
end_date = datetime.datetime.now().strftime("%d-%b-%Y")
end_date = str(end_date)

start_date = (datetime.datetime.now()- datetime.timedelta(days=days)).strftime("%d-%b-%Y")
start_date = str(start_date)

df=index_history("NIFTY 50",start_date,end_date)

df["state"]=df["CLOSE"].astype(float).pct_change()
df['state']=df['state'].apply(lambda x: 'Upside' if (x > 0) else 'Downside' )

df['priorstate']=df['state'].shift(1)

states = df [['priorstate','state']].dropna()
states_matrix = states.groupby(['priorstate','state']).size().unstack().fillna(0)

transition_matrix= states_matrix.apply(lambda x: x/float(x.sum()),axis=1)
t_0 = transition_matrix.copy()

t_m = t_0.copy()
t_n = t_0.dot(t_0)

i = 1
while(not(t_m.equals(t_n))):
    i += 1
    t_m = t_n.copy()
    t_n = t_n.dot(t_0)

print("Equilibrium Matrix Number: " + str(i))
print(t_n)

Equilibrium Matrix Number: 15
state       Downside    Upside
priorstate                    
Downside    0.538278  0.461722
Upside      0.538278  0.461722


So, t = 15, We get our equilibrium matrix. 

Anyways, the number of the matrix where we get the equilibrium does not matter much. What matters is the values. We can see the values are really close!

Interpreting, 

P(priorstate="Downside"/state="Downside) i.e. If We had a Downside day today, Tomorrow there is 53.6083% probability of having a Downside day.

Similarly, 

If We had a Downside day today, Tomorrow there is 46.17% probability of having an Upside day.